In [0]:
dbutils.widgets.text('src_table_name', '')
name=dbutils.widgets.get('src_table_name')

In [0]:
src_schema='silver'
src_tbl=name
trg_schema='gold'
dim_trg=f'dim_{src_tbl}'
surrogate_key=f'{dim_trg}key'
key_col = f'{src_tbl[:-1]}_id'
cdc='modified_date'

In [0]:
backdated_refresh=0


In [0]:
if backdated_refresh == 0:
    if spark.catalog.tableExists(f'Project.{trg_schema}.{dim_trg}'):
        last_load=spark.sql(f'''select max({cdc}) from project.`{trg_schema}`.`{dim_trg}`''')
        last_load=last_load.collect()[0][0]
    else:
        last_load='1900-01-01'
else:
    last_load=backdated_refresh

In [0]:
df_src = spark.sql(
    f'''SELECT * FROM project.`{src_schema}`.`{src_tbl}` where "{cdc}" > "{last_load}"'''
)

In [0]:
if spark.catalog.tableExists(f'Project.{trg_schema}.{dim_trg}'):
    df_sink=spark.sql(f'''select `{key_col}`,`{surrogate_key}`,created_date,updated_date from project.`{trg_schema}`.`{dim_trg}`''')
else:
    df_sink=spark.sql(f'''select '' as `{key_col}`,cast(null as int) as `{surrogate_key}`,cast(null as timestamp) as created_date,cast(null as timestamp) as updated_date where 1=0''')

In [0]:
from pyspark.sql.functions import *
df_joined=df_src.alias("df_src").join(df_sink.alias("sink"), on=key_col, how='left')\
.select(
        *[col(f"df_src.{c}") for c in df_src.columns],
        col(f"sink.{surrogate_key}"),
        col("sink.created_date"),
        col("sink.updated_date")
)

In [0]:
df_old=df_joined.filter(col(f"{surrogate_key}").isNotNull()).sort(col(f'{key_col}'))
df_new=df_joined.filter(col(f"{surrogate_key}").isNull()).sort(col(f'{key_col}'))

In [0]:
from pyspark.sql.functions import current_timestamp
df_old=df_old.withColumn('updated_date',current_timestamp())

In [0]:
if spark.catalog.tableExists(f'Project.{trg_schema}.{dim_trg}'):
    max_value=spark.sql(f'''select max({surrogate_key}) from project.{trg_schema}.{dim_trg}''')
    max_value=max_value.collect()[0][0]
else:
    max_value=0

In [0]:
from pyspark.sql.functions  import monotonically_increasing_id,current_timestamp
df_new=df_new.withColumn(f'{surrogate_key}', max_value+monotonically_increasing_id()+1)\
             .withColumn('created_date',current_timestamp())\
             .withColumn('updated_date', current_timestamp())


In [0]:
df_final_merge=df_new.union(df_old)

In [0]:
from delta.tables import DeltaTable

if spark.catalog.tableExists(f'Project.{trg_schema}.{dim_trg}'):
    df_trg = DeltaTable.forName(spark, f'Project.{trg_schema}.{dim_trg}')
    df_trg.alias('trg').merge(
        df_final_merge.alias('src'),
       f'trg.{surrogate_key} = src.{surrogate_key}'
    ).whenMatchedUpdateAll(condition=(f'src.updated_date>=trg.updated_date')) \
     .whenNotMatchedInsertAll() \
     .execute()
else:
    df_final_merge.write.format('delta').mode('overwrite').saveAsTable(f'Project.{trg_schema}.{dim_trg}')